In [4]:
import os
import io
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px
from IPython.display import display

# Display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 200)

# Path used in the environment (as provided)
default_path = '/content/Netflix Dataset.csv'

# Load dataset
if os.path.exists(default_path):
    df = pd.read_csv(default_path)
else:
    try:
        # If running in Colab, allow upload
        from google.colab import files
        print(f"File not found at {default_path}. Please upload the CSV file now (use the file chooser).")
        uploaded = files.upload()
        first_file = next(iter(uploaded.keys()))
        df = pd.read_csv(io.BytesIO(uploaded[first_file]))
    except Exception:
        raise FileNotFoundError(f"Dataset not found at {default_path}. Please upload the dataset or change the path.")

print('Dataset loaded. Shape:', df.shape)
print('Columns:')
print(list(df.columns))

display(df.head())

# %% [markdown]
## 2. Dataset Overview

# This section inspects dataset properties: missing values, datatypes, and summary statistics.

# %%
print('Info:')
print(df.info())

print('\nMissing values by column:')
print(df.isnull().sum())

print('\nBasic description:')
display(df.describe(include='all').T)

# %% [markdown]
## 3. Preprocessing

# %%
df.columns = [c.strip() for c in df.columns]
cols_lower = {c: c.lower().replace(' ', '_') for c in df.columns}
df.rename(columns=cols_lower, inplace=True)

print('\nAfter renaming, columns are:')
print(list(df.columns))

data = df.copy()

if 'release_year' in data.columns:
    data['release_year'] = pd.to_numeric(data['release_year'], errors='coerce')
else:
    possible_date_cols = [c for c in data.columns if 'date' in c or 'release' in c]
    if possible_date_cols:
        col = possible_date_cols[0]
        try:
            data['release_year'] = pd.to_datetime(data[col], errors='coerce').dt.year
        except Exception:
            data['release_year'] = pd.to_numeric(data[col].astype(str).str.extract(r'(\d{4})'), errors='coerce')
    else:
        data['release_year'] = np.nan

country_col = None
for cand in ['country', 'countries', 'country_of_origin']:
    if cand in data.columns:
        country_col = cand
        break

data['primary_country'] = data[country_col].astype(str).fillna('Unknown').apply(lambda x: x.split(',')[0].strip()) if country_col else 'Unknown'

genre_col = None
for cand in ['listed_in', 'genre', 'genres', 'type']:
    if cand in data.columns:
        genre_col = cand
        break

data['genres'] = data[genre_col].astype(str).fillna('Unknown') if genre_col else 'Unknown'

if 'type' not in data.columns:
    for cand in ['show_type', 'title_type', 'tv_show', 'type_of_show']:
        if cand in data.columns:
            data['type'] = data[cand]
            break
    else:
        if 'duration' in data.columns:
            data['type'] = data['duration'].astype(str).apply(lambda x: 'TV Show' if 'Season' in x or 'season' in x else 'Movie')
        else:
            data['type'] = 'Unknown'

display(data[['title','release_year','primary_country','genres','type']].head())

# %% [markdown]
## 4. Objective 1: Movies vs TV Shows over Years

# %%
trend = data.dropna(subset=['release_year']).groupby(['release_year','type']).size().reset_index(name='count')
trend = trend.sort_values(['release_year','type'])

print('Years covered:', int(data['release_year'].min()), 'to', int(data['release_year'].max()))

fig = px.line(trend, x='release_year', y='count', color='type', markers=True, title='Movies vs TV Shows Count Over Years')
fig.update_layout(xaxis_title='Year', yaxis_title='Count')
fig.show()

yearly_totals = data.dropna(subset=['release_year']).groupby('release_year').size().reset_index(name='total')
yearly_totals = yearly_totals.sort_values('release_year')
display(yearly_totals.head())

# %% [markdown]
## 5. Objective 2: Genre Trends

# %%
def explode_genres(df_in, col='genres'):
    s = df_in.assign(_genres=df_in[col].astype(str).str.split(',')).explode('_genres')
    s['_genres'] = s['_genres'].astype(str).str.strip()
    s.rename(columns={'_genres':'genre'}, inplace=True)
    return s

genres_exp = explode_genres(data, 'genres')

top_genres = genres_exp['genre'].value_counts().reset_index()
top_genres.columns = ['genre','count']
display(top_genres.head(20))

top_n = 10
topN = top_genres.head(top_n)['genre'].tolist()

genre_trend = genres_exp.dropna(subset=['release_year']).groupby(['release_year','genre']).size().reset_index(name='count')
genre_trend_top = genre_trend[genre_trend['genre'].isin(topN)]

fig2 = px.line(genre_trend_top, x='release_year', y='count', color='genre', markers=True, title=f'Trend of Top {top_n} Genres Over Years')
fig2.update_layout(xaxis_title='Year', yaxis_title='Count')
fig2.show()

# %% [markdown]
## 6. Objective 3: Country-wise Contributions

# %%
country_counts = data['primary_country'].value_counts().reset_index()
country_counts.columns = ['country','count']
display(country_counts.head(20))

top_countries = country_counts.head(8)['country'].tolist()
country_trend = data.dropna(subset=['release_year']).groupby(['release_year','primary_country']).size().reset_index(name='count')
country_trend_top = country_trend[country_trend['primary_country'].isin(top_countries)]

fig3 = px.line(country_trend_top, x='release_year', y='count', color='primary_country', markers=True, title='Top Countries Contribution Over Years')
fig3.update_layout(xaxis_title='Year', yaxis_title='Count')
fig3.show()

# %% [markdown]
## 7. Additional Summaries

# %%
type_dist = data['type'].value_counts().reset_index()
type_dist.columns = ['type','count']
display(type_dist)

if 'duration' in data.columns:
    mins = data['duration'].astype(str).str.extract(r'(\d+)')
    data['duration_mins'] = pd.to_numeric(mins[0], errors='coerce')
    display(data[['title','type','duration','duration_mins']].sort_values('duration_mins', ascending=False).head(20))

if 'director' in data.columns:
    top_directors = data['director'].astype(str).replace('nan','').str.split(',').explode().str.strip().value_counts().reset_index()
    top_directors.columns = ['director','count']
    display(top_directors.head(20))

# %% [markdown]
## 8. Expected Outcomes

# %%
outcomes = {
    'years_range': (int(data['release_year'].min()), int(data['release_year'].max())),
    'top_genres': top_genres.head(10).to_dict(orient='records'),
    'top_countries': country_counts.head(10).to_dict(orient='records'),
    'type_distribution': type_dist.to_dict(orient='records')
}

print('Expected Outcomes:')
print(outcomes)

# %% [markdown]
## 9. Save Results

# %%
output_dir = 'analysis_outputs'
os.makedirs(output_dir, exist_ok=True)

trend.to_csv(os.path.join(output_dir, 'movies_tv_trend_by_year.csv'), index=False)
genre_trend.to_csv(os.path.join(output_dir, 'genre_trend_by_year.csv'), index=False)
country_trend.to_csv(os.path.join(output_dir, 'country_trend_by_year.csv'), index=False)

print(f'CSV outputs saved in folder: {output_dir}')

# %% [markdown]
## End of Notebook


Dataset loaded. Shape: (7789, 11)
Columns:
['Show_Id', 'Category', 'Title', 'Director', 'Cast', 'Country', 'Release_Date', 'Rating', 'Duration', 'Type', 'Description']


,Show_Id,Category,Title,Director,Cast,Country,Release_Date,Rating,Duration,Type,Description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi",Brazil,"August 14, 2020",TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi & Fantasy","In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor."
1,s2,Movie,07:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, Azalia Ortiz, Octavio Michel, Carmen Beato",Mexico,"December 23, 2016",TV-MA,93 min,"Dramas, International Movies","After a devastating earthquake hits Mexico City, trapped survivors from all walks of life wait to be rescued while trying desperately to stay alive."
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence Koh, Tommy Kuan, Josh Lai, Mark Lee, Susan Leong, Benjamin Lim",Singapore,"December 20, 2018",R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow soldiers are forced to confront a terrifying secret that's haunting their jungle island training camp."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly, Christopher Plummer, Crispin Glover, Martin Landau, Fred Tatasciore, Alan Oppenheimer, Tom Kane",United States,"November 16, 2017",PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi & Fantasy","In a postapocalyptic world, rag-doll robots hide in fear from dangerous machines out to exterminate them, until a brave newcomer joins the group."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aaron Yoo, Liza Lapira, Jacob Pitts, Laurence Fishburne, Jack McGee, Josh Gad, Sam Golzari, Helen Carey, Jack Gilpin",United States,"January 1, 2020",PG-13,123 min,Dramas,A brilliant group of students become card-counting experts with the intent of swindling millions out of Las Vegas casinos by playing blackjack.


Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7789 entries, 0 to 7788
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Show_Id       7789 non-null   object
 1   Category      7789 non-null   object
 2   Title         7789 non-null   object
 3   Director      5401 non-null   object
 4   Cast          7071 non-null   object
 5   Country       7282 non-null   object
 6   Release_Date  7779 non-null   object
 7   Rating        7782 non-null   object
 8   Duration      7789 non-null   object
 9   Type          7789 non-null   object
 10  Description   7789 non-null   object
dtypes: object(11)
memory usage: 669.5+ KB
None

Missing values by column:
Show_Id            0
Category           0
Title              0
Director        2388
Cast             718
Country          507
Release_Date      10
Rating             7
Duration           0
Type               0
Description        0
dtype: int64

Basic description:


,count,unique,top,freq
Show_Id,7789,7787,s6621,2
Category,7789,2,Movie,5379
Title,7789,7787,The Lost Okoroshi,2
Director,5401,4050,"Raúl Campos, Jan Suter",18
Cast,7071,6831,David Attenborough,18
Country,7282,681,United States,2556
Release_Date,7779,1565,"January 1, 2020",118
Rating,7782,14,TV-MA,2865
Duration,7789,216,1 Season,1608
Type,7789,492,Documentaries,334



After renaming, columns are:
['show_id', 'category', 'title', 'director', 'cast', 'country', 'release_date', 'rating', 'duration', 'type', 'description']


...

,country,count
0,United States,2884
1,India,956
2,United Kingdom,577
3,nan,507
4,Canada,259
5,Japan,237
6,France,196
7,South Korea,194
8,Spain,168
9,Mexico,123


,type,count
0,Documentaries,334
1,Stand-Up Comedy,321
2,"Dramas, International Movies",320
3,"Comedies, Dramas, International Movies",243
4,"Dramas, Independent Movies, International Movies",215
...,...,...
487,"Crime TV Shows, International TV Shows, TV Sci-Fi & Fantasy",1
488,"Docuseries, Science & Nature TV, TV Action & Adventure",1
489,"British TV Shows, Classic & Cult TV, Kids' TV",1
490,"Docuseries, TV Sci-Fi & Fantasy",1


,title,type,duration,duration_mins
957,Black Mirror: Bandersnatch,"Dramas, International Movies, Sci-Fi & Fantasy",312 min,312
6852,The School of Mischief,"Comedies, Dramas, International Movies",253 min,253
4490,No Longer kids,"Comedies, Dramas, International Movies",237 min,237
3694,Lock Your Girls In,"Comedies, International Movies, Romantic Movies",233 min,233
5109,Raya and Sakina,"Comedies, Dramas, International Movies",230 min,230
5377,Sangam,"Classic Movies, Dramas, International Movies",228 min,228
3521,Lagaan,"Dramas, International Movies, Music & Musicals",224 min,224
3195,Jodhaa Akbar,"Action & Adventure, Dramas, International Movies",214 min,214
6509,The Irishman,Dramas,209 min,209
3282,Kabhi Khushi Kabhie Gham,"Dramas, International Movies",209 min,209


,director,count
0,,2388
1,Jan Suter,21
2,Raúl Campos,19
3,Marcus Raboy,16
4,Jay Karas,15
5,Cathy Garcia-Molina,13
6,Jay Chapman,12
7,Martin Scorsese,12
8,Youssef Chahine,12
9,Steven Spielberg,10


Expected Outcomes:
{'years_range': (2008, 2021), 'top_genres': [{'genre': 'International Movies', 'count': 2437}, {'genre': 'Dramas', 'count': 2108}, {'genre': 'Comedies', 'count': 1472}, {'genre': 'International TV Shows', 'count': 1199}, {'genre': 'Documentaries', 'count': 786}, {'genre': 'Action & Adventure', 'count': 721}, {'genre': 'TV Dramas', 'count': 704}, {'genre': 'Independent Movies', 'count': 675}, {'genre': 'Children & Family Movies', 'count': 532}, {'genre': 'Romantic Movies', 'count': 531}], 'top_countries': [{'country': 'United States', 'count': 2884}, {'country': 'India', 'count': 956}, {'country': 'United Kingdom', 'count': 577}, {'country': 'nan', 'count': 507}, {'country': 'Canada', 'count': 259}, {'country': 'Japan', 'count': 237}, {'country': 'France', 'count': 196}, {'country': 'South Korea', 'count': 194}, {'country': 'Spain', 'count': 168}, {'country': 'Mexico', 'count': 123}], 'type_distribution': [{'type': 'Documentaries', 'count': 334}, {'type': 'Stand-Up Co